In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
LETRAS_ROOT_URL = "https://www.letras.com"
LETRAS_REGGAETON_URL = LETRAS_ROOT_URL + "/mais-acessadas/reggaeton/"

response = requests.get(LETRAS_REGGAETON_URL)

soup = BeautifulSoup(response.text, "html.parser")

# Achicamos la búsqueda al contenedor
artists_container = soup.find("ol", {'class': 'top-list_art'})

In [3]:
# Como ya hemos achicado al contenedor, buscamos el patrón que se repite. En este caso las etiquetas a
artists_html = artists_container.find_all("a")

In [4]:
# Creamos la lista con las URL de los artistas
# artists_href = [artist.get('href') for artist in artists_html]
# artists_names = [artist.text for artist in artists_html]
artists_href = [[artist.get('href'), artist.text] for artist in artists_html]


# Imprimimos la lista de líneas. El pprint lo imprime con salto de línea
# pprint(artists_href)
df = pd.DataFrame(columns=['Artist', 'Title', 'Lyrics'])

In [5]:
for artist_href in artists_href:
    artist_href[0] = LETRAS_ROOT_URL + artist_href[0]

    art_response = requests.get(artist_href[0])
    art_soup = BeautifulSoup(art_response.text, "html.parser")

    # Achicamos la búsqueda al contenedor
    songs_container = art_soup.find("ol", {'class': 'cnt-list cnt-list--num cnt-list--col2'})
    
    # Como ya hemos achicado al contenedor, buscamos el patrón que se repite
    songs_html = songs_container.find_all("a")    #Creamos la lista con las líneas
    #songs_href = [song.get('href') for song in songs_html]
    #songs_titles = [song.text for song in songs_html]
    songs_href = [[song.get('href'), song.text] for song in songs_html]

    # Imprimimos las URL de cada una de las canciones de los artistas.
    # pprint(songs_href)

    for song_href in songs_href:
        song_href[0] = LETRAS_ROOT_URL + song_href[0]
        
        song_response = requests.get(song_href[0])
        song_soup = BeautifulSoup(song_response.text, "html.parser")

        # Achicamos la búsqueda al contenedor
        paragraphs_container = song_soup.find("div", {'class': 'cnt-letra p402_premium'})

        # Como ya hemos achicado al contenedor, buscamos el patrón que se repite
        paragraphs_html = paragraphs_container.find_all("p")    #Creamos la lista con las líneas
        lyrics = [paragraph.text for paragraph in paragraphs_html]

        # Imprimimos las URL de cada una de las canciones de los artistas.
        df = df.append({'Artist': artist_href[1], 'Title': song_href[1], 'Lyrics': lyrics} , ignore_index=True)
        # pprint(lyrics)

KeyboardInterrupt: 

In [6]:
df.head(50)

,Artist,Title,Lyrics
0,Karol G,Ocean,[Si algún día te vas de casa (de casa)Yo te ll...
1,Karol G,Dices Que Te Vas (part. Anuel AA),"[No sé, no-no séYo me despierto y busco tus me..."
2,Karol G,Love With a Quality (feat. Damian Marley),"[Chao, papaNow greetings to the worldVoice of ..."
3,Karol G,Punto G,"[(Dale, ey, ey), Sigue en la pista, me vas a e..."
4,Karol G,Créeme (part. Maluma),"[YayYaDice, ¿Cómo explicarle a la conciencia q..."
5,Karol G,Culpables (part. Anuel AA),"[Uah, uahMera, dime Karol G, bebesitaAnuel (oh..."
6,Karol G,A Ella,[Ella jugó a darte lo que más queríasYo jugué ...
7,Karol G,Mi Cama,[Dices que de mí ya te olvidasteY de tu mente ...
8,Karol G,Sin Corazón,[Yo no creo en el amor y tú no crees en el amo...
9,Karol G,Go Karo,"[O-O-Ovy On The Drums, Go, Karo!Enloqueció la ..."
